In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(0)

In [2]:
df = pd.read_csv(Path('./data/20201025.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice]
0,10/20/2020 17:33:32,NaN,NaN,NaN,NaN,NaN,OOPS ALL NICOLAS CAGE,PERVERTS: FRIEND OF FOE?,NaN,NaN,NaN,NaN
1,10/20/2020 17:48:23,The Ascent (1976),Heart of a Dog (1988),Valerie and her Week of Wonders (1970),The Witness (1969),For Eyes Only (1963),HALLOWEEN HANGOVER,PERVERTS: FRIEND OF FOE?,FROM SOUTH AMERICA,FUCKING VAMPIRES/VAMPIRES FUCKING,OOPS ALL NICOLAS CAGE,No Theme/Free-for-all
2,10/20/2020 17:51:34,The Ascent (1976),The Witness (1969),Valerie and her Week of Wonders (1970),Heart of a Dog (1988),For Eyes Only (1963),OOPS ALL NICOLAS CAGE,PERVERTS: FRIEND OF FOE?,HALLOWEEN HANGOVER,FUCKING VAMPIRES/VAMPIRES FUCKING,FROM SOUTH AMERICA,No Theme/Free-for-all
3,10/20/2020 18:07:53,For Eyes Only (1963),The Ascent (1976),Valerie and her Week of Wonders (1970),NaN,NaN,FROM SOUTH AMERICA,OOPS ALL NICOLAS CAGE,PERVERTS: FRIEND OF FOE?,FUCKING VAMPIRES/VAMPIRES FUCKING,HALLOWEEN HANGOVER,NaN
4,10/20/2020 18:08:57,For Eyes Only (1963),The Ascent (1976),Valerie and her Week of Wonders (1970),The Witness (1969),Heart of a Dog (1988),OOPS ALL NICOLAS CAGE,PERVERTS: FRIEND OF FOE?,HALLOWEEN HANGOVER,FUCKING VAMPIRES/VAMPIRES FUCKING,FROM SOUTH AMERICA,No Theme/Free-for-all
5,10/20/2020 22:11:49,The Witness (1969),The Ascent (1976),Heart of a Dog (1988),Valerie and her Week of Wonders (1970),For Eyes Only (1963),FUCKING VAMPIRES/VAMPIRES FUCKING,PERVERTS: FRIEND OF FOE?,OOPS ALL NICOLAS CAGE,HALLOWEEN HANGOVER,FROM SOUTH AMERICA,No Theme/Free-for-all
6,10/21/2020 8:00:59,Valerie and her Week of Wonders (1970),For Eyes Only (1963),The Ascent (1976),Heart of a Dog (1988),The Witness (1969),OOPS ALL NICOLAS CAGE,PERVERTS: FRIEND OF FOE?,HALLOWEEN HANGOVER,FUCKING VAMPIRES/VAMPIRES FUCKING,FROM SOUTH AMERICA,NaN


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['The Ascent (1976)', 'Heart of a Dog (1988)',
       'Valerie and her Week of Wonders (1970)', 'The Witness (1969)',
       'For Eyes Only (1963)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'The Ascent (1976)': <Candidate('The Ascent (1976)')>,
 'Heart of a Dog (1988)': <Candidate('Heart of a Dog (1988)')>,
 'Valerie and her Week of Wonders (1970)': <Candidate('Valerie and her Week of Wonders (1970)')>,
 'The Witness (1969)': <Candidate('The Witness (1969)')>,
 'For Eyes Only (1963)': <Candidate('For Eyes Only (1963)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot()>,
 <Ballot(The Ascent (1976), Heart of a Dog (1988), Valerie and her Week of Wonders (1970), The Witness (1969), For Eyes Only (1963))>,
 <Ballot(The Ascent (1976), The Witness (1969), Valerie and her Week of Wonders (1970), Heart of a Dog (1988), For Eyes Only (1963))>,
 <Ballot(For Eyes Only (1963), The Ascent (1976), Valerie and her Week of Wonders (1970))>,
 <Ballot(For Eyes Only (1963), The Ascent (1976), Valerie and her Week of Wonders (1970), The Witness (1969), Heart of a Dog (1988))>,
 <Ballot(The Witness (1969), The Ascent (1976), Heart of a Dog (1988), Valerie and her Week of Wonders (1970), For Eyes Only (1963))>,
 <Ballot(Valerie and her Week of Wonders (1970), For Eyes Only (1963), The Ascent (1976), Heart of a Dog (1988), The Witness (1969))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                                 Votes  Status
--------------------------------------  -------  --------
The Ascent (1976)                             2  Hopeful
For Eyes Only (1963)                          2  Hopeful
The Witness (1969)                            1  Rejected
Valerie and her Week of Wonders (1970)        1  Rejected
Heart of a Dog (1988)                         0  Rejected
Blank votes                                   1  Rejected

FINAL RESULT
Candidate                                 Votes  Status
--------------------------------------  -------  --------
The Ascent (1976)                             3  Elected
For Eyes Only (1963)                          3  Rejected
The Witness (1969)                            0  Rejected
Valerie and her Week of Wonders (1970)        0  Rejected
Heart of a Dog (1988)                         0  Rejected
Blank votes                                   1  Rejected



In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                            Votes  Status
---------------------------------  -------  --------
OOPS ALL NICOLAS CAGE                    4  Hopeful
FROM SOUTH AMERICA                       1  Hopeful
HALLOWEEN HANGOVER                       1  Rejected
FUCKING VAMPIRES/VAMPIRES FUCKING        1  Rejected
PERVERTS: FRIEND OF FOE?                 0  Rejected
No Theme/Free-for-all                    0  Rejected

FINAL RESULT
Candidate                            Votes  Status
---------------------------------  -------  --------
OOPS ALL NICOLAS CAGE                    5  Elected
FROM SOUTH AMERICA                       2  Rejected
HALLOWEEN HANGOVER                       0  Rejected
FUCKING VAMPIRES/VAMPIRES FUCKING        0  Rejected
PERVERTS: FRIEND OF FOE?                 0  Rejected
No Theme/Free-for-all                    0  Rejected

